In [ ]:
#合併資料
import glob

# 讀取 item_category 資料集
item_category_path = "C:/Users/b0983/OneDrive/桌面/ADENX/item_category.csv" 
item_category_data = pd.read_csv(item_category_path)

# 讀取 Orders 和 Member 資料集
orders = pd.read_csv('C:/Users/b0983/OneDrive/桌面/ADENX/orders.csv')
members = pd.read_csv('C:/Users/b0983/OneDrive/桌面/ADENX/member.csv')

# 合併 Orders 和 Members 資料集
orders_members = orders.merge(members, on='customer_id', how='left')


# 將 orders_members 中的 sku 轉換為商品名稱
orders_members = orders_members.merge(item_category_data, on='sku', how='left')

# 目前日期
current_date = datetime(2024, 6, 19)

# 定義計算年齡的函數
def calculate_age(birthday):
    if pd.isna(birthday):
        return None
    birth_date = datetime.strptime(birthday, '%Y-%m-%d')
    age = current_date.year - birth_date.year - ((current_date.month, current_date.day) < (birth_date.month, birth_date.day))
    return int(age)

# 計算 age 並新增 age 欄位
orders_members['age'] = orders_members['birthday'].apply(calculate_age)

# 檢查結果
print(orders_members[['birthday', 'age']].head())
display(orders_members)

In [ ]:
#處理合併後資料，挑選高級會員
import pandas as pd
from datetime import datetime

# 讀取上傳的文件
file_path = 'your_path/orders_members.csv'
orders_members = pd.read_csv(file_path)

# 高級會員資料處理
high_members = orders_members[(orders_members['membership_tier'] == '銀級會員') | (orders_members['membership_tier'] == '金鑽會員')]

# 將 cutoff_date 設置為時區感知的日期時間對象
cutoff_date = pd.Timestamp('2024-01-01', tz='UTC')

# 確保日期欄位為日期時間格式
high_members['registered_at'] = pd.to_datetime(high_members['registered_at'], utc=True)
high_members['created_at'] = pd.to_datetime(high_members['created_at'], utc=True)

# 篩選註冊滿一年的銀級會員
high_members_one_year = high_members[high_members['registered_at'] <= cutoff_date - pd.DateOffset(years=1)]

# 去除相同 order_number 的重複訂單
high_members_one_year_unique_orders = high_members_one_year.drop_duplicates(subset='order_number')

# 設定分析日期
analysis_date = pd.Timestamp('2023-12-31', tz='UTC')
start_date = pd.Timestamp('2023-01-01', tz='UTC')

# 計算一年內有消費和一年內未消費的顧客
one_year_orders = high_members_one_year_unique_orders[(high_members_one_year_unique_orders['created_at'] >= start_date) & (high_members_one_year_unique_orders['created_at'] <= analysis_date)]
one_year_customers = one_year_orders['customer_id'].unique()
one_year_no_purchase_customers = high_members_one_year_unique_orders[~high_members_one_year_unique_orders['customer_id'].isin(one_year_customers)]['customer_id'].unique()

# 計算消費次數和購物週期
consumption_counts = one_year_orders.groupby('customer_id')['order_number'].count()
one_year_orders = one_year_orders.sort_values(['customer_id', 'created_at'])
one_year_orders['previous_order_date'] = one_year_orders.groupby('customer_id')['created_at'].shift(1)
one_year_orders['days_between_orders'] = (one_year_orders['created_at'] - one_year_orders['previous_order_date']).dt.days

average_shopping_cycle = one_year_orders.groupby('customer_id')['days_between_orders'].mean().dropna()
shopping_cycle_median = average_shopping_cycle.median()

print("銀級會員平均購買頻率（購物週期）的中位數:", shopping_cycle_median)

# 針對第一群顧客的分群
def categorize_active_customer(row, shopping_cycle_median, analysis_date):
    consumption_count = row['consumption_count']
    last_purchase_date = row['last_purchase_date']
    
    if consumption_count == 1 and (analysis_date - last_purchase_date).days < shopping_cycle_median:
        return 'N'  # 一年內第一次購物的顧客
    elif consumption_count > 1 and (analysis_date - last_purchase_date).days < shopping_cycle_median:
        return 'A'  # 一年內有兩次購物以上的顧客
    elif consumption_count == 1 and (analysis_date - last_purchase_date).days >= shopping_cycle_median:
        return 'L'  # 一年內買過一次但超過中位數天沒買第二次的顧客
    elif consumption_count > 1 and (analysis_date - last_purchase_date).days >= shopping_cycle_median:
        return 'P'  # 曾經買過2次以上，但近中位數天都沒有購物的顧客
    else:
        return 'Other1'

active_df = pd.DataFrame({
    'consumption_count': consumption_counts,
    'shopping_cycle': average_shopping_cycle,
    'last_purchase_date': one_year_orders.groupby('customer_id')['created_at'].max()
})

active_df['category'] = active_df.apply(categorize_active_customer, axis=1, shopping_cycle_median=shopping_cycle_median, analysis_date=analysis_date)

# 針對第二群顧客的分群
def categorize_inactive_customer(row, analysis_date):
    last_purchase_date = row['last_purchase_date']
    
    if row['total_orders'] == 0:
        return 'R'  # 註冊後從來沒有消費過的顧客
    elif (analysis_date - last_purchase_date).days >= 364:
        return 'S'  # 已經超過一年沒購買過的顧客
    else:
        return 'Other2'

inactive_customers_df = high_members_one_year_unique_orders[high_members_one_year_unique_orders['customer_id'].isin(one_year_no_purchase_customers)]
total_orders = high_members_one_year_unique_orders.groupby('customer_id')['order_number'].count()
inactive_df = inactive_customers_df.groupby('customer_id').agg({'created_at': 'max'}).reset_index()
inactive_df.columns = ['customer_id', 'last_purchase_date']
inactive_df['total_orders'] = inactive_df['customer_id'].map(total_orders)

inactive_df['category'] = inactive_df.apply(categorize_inactive_customer, axis=1, analysis_date=analysis_date)

# 合併兩群顧客的分群結果
final_df = pd.concat([active_df, inactive_df.set_index('customer_id')], axis=0)

# 計算銀級會員2022年的消費總次數和消費總金額及總數量
silver_members_2022 = high_members[high_members['created_at'].dt.year == 2022]
order_count = silver_members_2022.groupby('customer_id')['order_number'].nunique().reset_index()
order_sum = silver_members_2022.groupby('customer_id')['items_total_dollar_after_discount'].sum().reset_index()
quantity_sum = silver_members_2022.groupby('customer_id')['quantity'].sum().reset_index()

# 合併計算結果
summary = pd.merge(order_count, order_sum, on='customer_id')
summary = pd.merge(summary, quantity_sum, on='customer_id')

summary.columns = ['customer_id', 'total_orders_2022', 'total_amount_2022', 'total_quantity_2022']

# 合併顧客分群結果與先前的計算結果
final_df = final_df.reset_index()
final_df = final_df.merge(summary, on='customer_id')

# 顯示結果
display(final_df)


In [ ]:
#選取某年進行分析
# 將 "created_at" 轉換為日期時間格式
orders_members['created_at'] = pd.to_datetime(orders_members['created_at'], errors='coerce')

# 篩選出 "created_at" 在 2022 年的資料
filtered_data = orders_members[orders_members['created_at'].dt.year == 2022]
# 移除不需要的列
filtered_data = orders_members.drop(columns=['created_at', 'updated_at', 'sku', 'item_variation_id_x','item_variation_id_y', 'birthday','registered_at'])

categoried_dataset = pd.merge(filtered_data, final_df[['customer_id', 'consumption_count', 'shopping_cycle',
       'last_purchase_date', 'category', 'total_orders_2022',
       'total_amount_2022', 'total_quantity_2022']], on='customer_id', how='left')

# 高級會員資料處理
high_members = categoried_dataset[(categoried_dataset['membership_tier'] == '銀級會員') | (categoried_dataset['membership_tier'] == '金鑽會員')]

# 顯示結果
display(high_members)

In [ ]:
#進行缺失值填補
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix

# 定義自變數和反應變數
variables = ['created_by_x', 'channel_created_by_channel_name', 'order_payment_type', 'item_count',
             'quantity', 'item_normal_price', 'item_price_sale', 'items_total_dollar',
             'discounted_price_dollars', 'items_total_dollar_after_discount', 'created_by_y',
             'gender', 'is_sms', 'is_email', 'is_line', 'item_category', 'age',
             'consumption_count', 'shopping_cycle', 'total_orders_2022', 'total_amount_2022',
             'total_quantity_2022']

# 定義自變數和反應變數
X = high_members[variables]
Y = high_members['category']

# 檢查數據中是否存在NaN值，並填補數據中的NaN值
numerical_features = ['age', 'quantity', 'item_count', 'item_normal_price', 'item_price_sale', 'items_total_dollar', 
                      'discounted_price_dollars', 'items_total_dollar_after_discount', 'consumption_count', 
                      'shopping_cycle', 'total_orders_2022', 'total_amount_2022', 'total_quantity_2022']

categorical_features = ['created_by_x', 'created_by_y', 'gender', 'is_sms', 'is_email', 'is_line', 
                        'channel_created_by_channel_name', 'order_payment_type', 'item_category']

# 使用SimpleImputer進行數值型和類別型特徵的缺失值填補
numerical_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

X[numerical_features] = numerical_imputer.fit_transform(X[numerical_features])
X[categorical_features] = categorical_imputer.fit_transform(X[categorical_features])

# 確保沒有 NaN 值存在於數據中
X = X.fillna(0)
Y = Y.fillna('Unknown')

# 打印唯一的分類
print("唯一的分類:", Y.unique())

# 過濾掉不需要的分類，只保留N, A, L, P, S
valid_classes = ['N', 'A', 'L', 'P', 'S']
filtered_indices = Y.isin(valid_classes)
X = X[filtered_indices]
Y = Y[filtered_indices]

# 再次檢查是否還存在NaN值
print("數據中是否還存在NaN值: \n", X.isna().sum())
print("反應變數中是否還存在NaN值: \n", Y.isna().sum())


In [ ]:
#進行隨機森林建模
# 定義數據預處理流水線
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 建構隨機森林模型流水線
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# 分割資料集為訓練集和測試集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 訓練模型
pipeline.fit(X_train, Y_train)

# 預測測試集
Y_pred = pipeline.predict(X_test)

# 評估模型
classification_report_result = classification_report(Y_test, Y_pred, target_names=valid_classes)
confusion_matrix_result = confusion_matrix(Y_test, Y_pred)
print("分類報告:\n", classification_report_result)
print("混淆矩陣:\n", confusion_matrix_result)

In [ ]:
# 使用5折交叉驗證評估模型
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(pipeline, X, Y, cv=5, scoring='accuracy')
print(f"交叉驗證準確性: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}")

In [ ]:
#顯示模型重要特徵
import matplotlib.pyplot as plt

# 提取數值型和類別型特徵名稱
numerical_feature_names = numerical_features
categorical_feature_names = pipeline.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_features).tolist()

# 合併所有特徵名稱
feature_names = numerical_feature_names + categorical_feature_names

# 提取特徵重要性
importances = pipeline.named_steps['classifier'].feature_importances_

# 創建特徵重要性數據框
feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importances})

# 排序特徵重要性
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# 只顯示前10個最重要的特徵
top_features = feature_importances.head(15)

# 顯示特徵重要性數據框
print("前15個特徵重要性表:")
print(top_features)

# 繪製特徵重要性圖
plt.figure(figsize=(10, 6))
plt.barh(top_features['feature'], top_features['importance'])
plt.xlabel('Importance')
plt.title('Top 15 Feature Importances')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# 針對銀級會員分析
sliver_members = categoried_dataset[categoried_dataset['membership_tier'] == '銀級會員']
#查看各分群各性別之消費力
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
# 設定中文字體
font_path = 'C:/Windows/Fonts/msjh.ttc'  # 微軟正黑體的字體路徑
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams['axes.unicode_minus'] = False  # 解決負號顯示問題

# 確保數據不包含 NaN 值
sliver_members = sliver_members.dropna(subset=['items_total_dollar_after_discount', 'item_count', 'gender', 'category'])

# 定義分群標籤
categories = ['N', 'A', 'P', 'L', 'S']

for category in categories:
    # 選擇當前分群資料
    category_data = sliver_members[sliver_members['category'] == category]

    # 繪製散點圖
    plt.figure(figsize=(10, 6))

    # 女性資料點
    female_data = category_data[category_data['gender'] == 'female']
    plt.scatter(female_data['items_total_dollar_after_discount'], female_data['item_count'], color='red', label='Female', alpha=0.6)

    # 男性資料點
    male_data = category_data[category_data['gender'] == 'male']
    plt.scatter(male_data['items_total_dollar_after_discount'], male_data['item_count'], color='blue', label='Male', alpha=0.6)

    # 添加標題和標籤
    plt.title(f'購買金額與購買商品數（分群: {category}）')
    plt.xlabel('Items Total Dollar After Discount')
    plt.ylabel('Item Count')
    plt.legend()

    # 顯示圖表
    plt.show()

In [ ]:
#繪製各分群付款方式散布圖
# 定義不同付款方式的顏色
payment_type_colors = {
    'pos': 'blue',
    'linepay': 'green',
    'shopline_payment': 'purple',
    'taishin': 'orange',
    'tw_tcat_cod': 'red',
    'neweb_pay_v2': 'cyan',
    'custom': 'magenta',
    'cash_on_delivery': 'brown',
    'free_checkout': 'grey',
    'taishin_subscription': 'yellow'
}

for category in categories:
    # 選擇當前分群資料
    category_data = sliver_members[sliver_members['category'] == category]

    # 繪製散點圖
    plt.figure(figsize=(10, 6))

    # 根據付款方式繪製資料點
    for payment_type, color in payment_type_colors.items():
        payment_data = category_data[category_data['order_payment_type'] == payment_type]
        plt.scatter(payment_data['items_total_dollar_after_discount'], payment_data['item_count'], 
                    color=color, label=payment_type, alpha=0.6)

    # 添加標題和標籤
    plt.title(f'購買金額與購買商品數（分群: {category}）', fontproperties=font_prop)
    plt.xlabel('Items Total Dollar After Discount', fontproperties=font_prop)
    plt.ylabel('Item Count', fontproperties=font_prop)
    plt.legend(title='Payment Type')

    # 顯示圖表
    plt.show()

In [ ]:
# 繪製每個顧客分群的訂單渠道分佈
# 過濾掉缺少 category 或 channel 的數據，並且移除 category 為 "Other2" 的行
filtered_sliver = sliver_members.dropna(subset=['category', 'channel_created_by_channel_name'])
filtered_sliver = filtered_sliver[filtered_sliver['category'] != 'Other2']
# 計算不同 category 和 channel 的分佈
channel_composition = filtered_sliver.groupby(['category', 'channel_created_by_channel_name']).size().unstack().fillna(0)

for category in categories:
    plt.figure(figsize=(12, 8))
    category_data = channel_composition.loc[category]

    # 排序數據
    category_data = category_data.sort_values(ascending=False)

    # 繪製長條圖
    category_data.plot(kind='bar', colormap='viridis', alpha=0.7)

    # 添加圖標和標題
    plt.title(f'銀級會員的訂單渠道分佈（分群: {category}）', fontproperties=font_prop)
    plt.xlabel('訂單渠道', fontproperties=font_prop)
    plt.ylabel('人數', fontproperties=font_prop)
    plt.xticks(rotation=45, fontproperties=font_prop)
    plt.legend(title='Channel', prop=font_prop)
    plt.tight_layout()

    # 顯示圖表
    plt.show()

In [ ]:
# 繪製每個顧客分群的長條圖並按照銷售量高低排列
# 計算各顧客分群購買的商品類別總數
category_item_count = filtered_data.groupby(['category', 'item_category'])['item_count'].sum().unstack().fillna(0)

for category in categories:
    plt.figure(figsize=(12, 8))
    category_data = category_item_count.loc[category].sort_values(ascending=False)
    category_data.plot(kind='bar')
    
    # 添加圖標和標題
    plt.title(f'顧客分群 {category} 購買的商品類別總數')
    plt.xlabel('商品類別 (Item Category)')
    plt.ylabel('商品數量 (Item Count)')
    plt.tight_layout()
    
    # 顯示圖表
    plt.show()

In [ ]:
# 繪製每個顧客分群的長條圖並按照銷售量高低排列
# 計算各顧客分群和性別購買的商品類別總數
category_item_count_gender = filtered_data.groupby(['category', 'item_category', 'gender'])['item_count'].sum().unstack().fillna(0)

for category in categories:
    plt.figure(figsize=(12, 8))
    category_data = category_item_count_gender.loc[category]
    category_data = category_data.loc[category_data.sum(axis=1).sort_values(ascending=False).index]
    category_data.plot(kind='bar', stacked=False, alpha=0.7, colormap='Set1')
    
    # 添加圖標和標題
    plt.xlabel('商品類別 (Item Category)')
    plt.ylabel('商品數量 (Item Count)')
    plt.legend(title='性別 (Gender)')
    plt.tight_layout()
    
    # 顯示圖表
    plt.show()